# faimed3d layers and functions

In [1]:
# hide
import sys
sys.path.append("..")

In [2]:
# export
# default_exp layers
from faimed3d.basics import *
from faimed3d.augment import *
from faimed3d.preprocess import *
from faimed3d.data import *
from fastai.vision.learner import _default_meta, _add_norm, model_meta, create_body
from fastai.layers import ResBlock

In [3]:
# export
from fastai.basics import *
from fastai.callback.all import *

In [4]:
d = pd.read_csv('../data/radiopaedia_cases.csv')
dls = ImageDataLoaders3D.from_df(d, 
                                 item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (20, 150, 150), perc_crop = True),
                                 bs = 2, 
                                 val_bs = 2)

## Helper functions
Some functions from `fastai.layers` are needed to construct Learners (see next notebook). For this some slight modifications had to be made. 
The `in_channel` function had to be modified to also accept 3D models wich have 5D weights and the `num_features_model` function was adapted to pass a size tuple of len 3 instead of 2. The other functions were not changed but copied to avoid conflicts when loaded directly from fastai.   
`cnn_learner_3d` is essentially the same function as fastais `cnn_learner`, just adds a new callback. 

`in_channels` form fastai only returns a result if `weight.ndim == 4` but in 3D convolutional layers, it will be 5 dimensions, so the functions has to be adapted. 

In [5]:
# export
def in_channels(m):
    """
    Return the shape of the first weight layer in `m`.
    same as fastai.vision.learner.in_channels but allows l.weight.ndim of 4 and 5
    """
    for l in flatten_model(m):
        if getattr(l, 'weight', None) is not None and l.weight.ndim in [4,5]:
            return l.weight.shape[1]
    raise Exception('No weight layer')

`num_features_model` is unchanged, but needs to be defined here to correctly call the adapted `in_channels` function

In [6]:
# export
def num_features_model(m):
    """
    Return the number of output features for `m`.
    same as fastai.vision.learner.num_features_model passes model_size a len 3 tuple of sz

    """
    sz,ch_in = 32,in_channels(m)
    while True:
        #Trying for a few sizes in case the model requires a big input size.
        try:
            return model_sizes(m, (sz,sz,sz))[-1][1]
        except Exception as e:
            sz *= 2
            print(sz)
            if sz > 2048: raise e

`create_cnn_model` is unchanged, but needs to be redefined to correctly call `num_features_model` which then calls the changed `in_channels` function

In [7]:
# export
def create_cnn_model(arch, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom convnet architecture using `arch`, `n_in` and `n_out`. Identical to fastai func"
    body = create_body(arch, n_in, pretrained, cut)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

`model_sizes` and `dummy_eval` are identical to `fastai` versions, but needed to be redefined so that the correct `in_channels` function is called

In [8]:
# export
def model_sizes(m, size=(64,64)):
    "Pass a dummy input through the model `m` to get the various sizes of activations. same as fastai func"
    with hook_outputs(m) as hooks:
        _ = dummy_eval(m, size=size)
        return [o.stored.shape for o in hooks]

In [9]:
# export
def dummy_eval(m, size=(64,64)):
    "Evaluate `m` on a dummy input of a certain `size`. Same as fastai func"
    ch_in = in_channels(m)
    x = one_param(m).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)
    with torch.no_grad(): return m.eval()(x)

`fastai` performes adaptive concat pooling as first step in the new header, which is adapted to 3D. 

In [10]:
# export
class AdaptiveConcatPool3d(Module):
    "Layer that concats `AdaptiveAvgPool3d` and `AdaptiveMaxPool3d`"
    def __init__(self, size=None):
        self.size = size or 1
        self.ap = nn.AdaptiveAvgPool3d(self.size)
        self.mp = nn.AdaptiveMaxPool3d(self.size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

## Wrapper for 4D Models

In radiology, often multiple sequences are needed for a diagnosis. For example, while viewing a head MRI DWI and ADC map have to be viewed together for a stroke diagnosis and to advoid false diagnosis from T2 shine trough. The data effectively becomes 4D. There are no modules for 4D convolution, so a workarround needs to be defined which processes the multiple 3D volumes after each other and then pools the information. The models should also still work with available pretrained 3D models, so no completly new architectures are defined in `faimed3d`. 

When using multiple inputs, `nn.Sequential` will not work as it expects only two inputs (self and input). Therefore a subclass is defined, which accepts multiple inputs and converts those into a tuple, which is then passed to the modules. 

In [11]:
# export
class Sequential4D(nn.Sequential):
    def forward(self, *input):
        input = tuple(input)
        for module in self:
            input = module(input)
        return input

The `Arch4D` is designed to recieve a (pretrained) model arch from the fastai function `create_body`. It assumes, that the arch is buil following the common structure of PyTorch models. That means it consists out of multiple submodules, where the first is a stem.  
For each element in the input, the stem is copied so that the weights of the different stems can differ. After an element of input is passed through the stem, it is further processed by the `main_body`. So the inputs are each passed thorugh an individual CNNs which share all weights except for the stem. The output of each individual CNN is captured and stored in a list which is eventually concatenated to a single tensor. 

In [12]:
# export
class Arch4D(nn.Module):
    def __init__(self, arch, n_inp):
        super(Arch4D, self).__init__()
        
        for i in range(0, n_inp):
            setattr(self, 'BasicStem_{}'.format(i), arch[0])
        self.main_body = arch[1:]

    def forward(self, inp):
        out = []
        for i, x in enumerate(inp):
            x = getattr(self, 'BasicStem_{}'.format(i))(x)
            out.append(self.main_body(x))
        return torch.cat(out, 1)
            

When `fastai` creates a new model head, first the input is pooled with `AdaptiveConcatPool`, then flattened ans passed trough two linear layers might not be the best approach for multiple inputs, as the first layer would recieve `n_inp` * `n_features` inputs and reduces it to 512 features. So the first linear layer wil much more reduce the feature information than the second. So in `faimed3d` an additional convolutional layer with kernel size 1 and stride 1 is used to pool the number of features from `n_features` * `n_inp` to `n_features`. Then the normal `fastai` head is added. We added the last convolutional layer to the head, so that the freeze and unfreeze operations of `fastai` still work as expected. 

In [13]:
# export
def create_head(nf, n_out, n_inp=None, lin_ftrs=None, ps=0.5, concat_pool=True, bn_final=False, lin_first=False, y_range=None):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    pool = AdaptiveConcatPool3d() if concat_pool else nn.AdaptiveAvgPool3d(1)
    if n_inp is not None: 
        pool = nn.Sequential(pool, 
                             ConvLayer(n_inp*nf, nf, ndim = 3, ks = 1, stride = 1))
    layers = [pool, Flatten()]
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    if y_range is not None: layers.append(SigmoidRange(*y_range))
    return nn.Sequential(*layers)

`create_cnn_model_4d` is similar to `create_cnn_model` but expects `n_inp` as additional argument. Depending on `n_inp` the respective network is constructed. 

In [14]:
# export
def create_cnn_model_4d(arch, n_out, n_inp, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom convnet architecture using `arch`, `n_in` and `n_out`. Identical to fastai func"
    body = create_body(arch, n_in, pretrained, cut)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, n_inp, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    body = Arch4D(body, n_inp)
    model = Sequential4D(body, head)
    if init is not None: apply_init(model[1], init)
    return model

In [15]:
from torchvision.models.video import r3d_18
model = create_cnn_model_4d(r3d_18, 2, 2)
model(torch.randn(2, 3, 3, 10, 10), torch.randn(2, 3, 3, 10, 10)).size()

torch.Size([2, 2])

In [16]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_transforms.ipynb.
Converted 04_dataloaders.ipynb.
Converted 05_layers.ipynb.
Converted 06_learner.ipynb.
Converted 06a_models.alexnet.ipynb.
Converted 06b_models.resnet.ipynb.
Converted 06d_models.unet.ipynb.
Converted 06f_models.losses.ipynb.
Converted 07_callback.ipynb.
Converted index.ipynb.


###### 